<a href="https://colab.research.google.com/github/guilhermelaviola/IntelligentCommunication/blob/main/Class11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **API Security and Authentication**
API security is vital for protecting data and ensuring only authorized access. Authentication verifies user identity, while authorization defines resource access levels, often using JWT tokens and OAuth 2.0. Cyberattack defenses include mitigating injection attacks through input validation and DDoS attacks with rate limits. Implementing HTTPS ensures encrypted data transmission, enhancing user confidence. Continuous monitoring and auditing of API activity are essential for identifying performance issues and detecting unauthorized access. A comprehensive API security strategy encompasses multiple layers and requires ongoing vigilance to adhere to best practices and protection mechanisms.

In [1]:
# Importing all the necessary libraries:
import http.server
import json
import ssl
from urllib.parse import urlparse
import requests
from base64 import b64decode
from http import HTTPStatus

In [2]:
# API configuration:
GOOGLE_API_KEY = 'YOUR_GOOGLE_API_KEY'  # Replace with your API key

In [3]:
# Class to manage the API:
class MyRequestHandler(http.server.BaseHTTPRequestHandler):

  def do_GET(self):
    if self.path == '/protected':
      if self.authenticate():
        self.send_response(HTTPStatus.OK)
        self.send_header('Content-type', 'application/json')
        self.end_headers()
        response = {'message': 'Protected content'}
        self.wfile.write(json.dumps(response).encode())
      else:
        self.send_response(HTTPStatus.UNAUTHORIZED)
        self.end_headers()
        response = {'message': 'Unauthorized'}
        self.wfile.write(json.dumps(response).encode())
      else:
        self.send_response(HTTPStatus.NOT_FOUND)
        self.end_headers()

      def do_POST(self):
        if self.path == '/chat':
          content_length = int(self.headers['Content-Length'])
          post_data = self.rfile.read(content_length)
          data = json.loads(post_data)

          user_message = data.get('message')
          bot_response = self.get_google_ai_response(user_message)

          self.send_response(HTTPStatus.OK)
          self.send_header('Content-type', 'application/json')
          self.end_headers()
          self.wfile.write(json.dumps({'response': bot_response}).encode())

        else:
          self.send_response(HTTPStatus.NOT_FOUND)
          self.end_headers()

        def authenticate(self):
            auth_header = self.headers.get('Authorization')
            if auth_header:
              try:
                auth_type, credentials = auth_header.split()
                if auth_type.lower() == 'basic':
                  decoded_credentials = b64decode(credentials).decode('utf-8')
                  username, password = decoded_credentials.split(':')
                  return username == 'user1' and password == 'password1'
                except Exception:
                return False
              return False

        def get_google_ai_response(self, user_message):
          url = 'https://google.generativeai.googleapis.com/v1beta2/chat.completions:generate'
          headers = {
              'Content-Type': 'application/json',
              'Authorization': f'Bearer {GOOGLE_API_KEY}'
              }
              body = {
                  'messages': [{'role': 'user', 'content': user_message}],
                  'model': 'gemini-1.5-pro-latest'  # Change it accordingly
                  }

              try:
                response = requests.post(url, headers=headers, json=body)
                response.raise_for_status()  # Throws an error as error response
                return response.json().get('choices', [{}])[0].get('message', {}).get('content', 'No response')
              except requests.RequestException as e:
                return f'Error establishing communication with Google API: {str(e)}'

# Function to start the server:
def run(server_class=http.server.HTTPServer, handler_class=MyRequestHandler):
  server_address = ('', 8080)
  httpd = server_class(server_address, handler_class)

  # SSL Configuration:
  httpd.socket = ssl.wrap_socket(httpd.socket,
                                 keyfile='path/to/your/server-key.pem',  # Insert the path for your private key file.
                                 certfile='path/to/your/server-cert.pem',  # Insert the path for your certificate.
                                 server_side=True)

  print('Starting server on https://localhost:8080...')
  httpd.serve_forever()

if __name__ == '__main__':
  run()

SyntaxError: invalid syntax (ipython-input-159881232.py, line 17)